In [136]:
# Dependencies
import json
import requests 
import pandas as pd
from pprint import pprint

In [137]:
# Specify the API URL
url = "http://127.0.0.1:5000/api/data"

# Make request and store response
response = requests.get(url)
response_json = response.json()

In [138]:
# Save information to dataframe
summary = pd.DataFrame(response_json)

In [139]:
# Create a dictionary of stations
destinationsUnclean = summary.drop_duplicates(subset=["End_station", "End_station_number"])
destinationsUnclean2 = destinationsUnclean.drop(columns=["Bike_model", "Bike_number", "End_date", "Number", "Start_date", "Start_lat", "Start_lon", "Start_station", "Start_station_number", "Total_duration", "Total_duration_ms"])
destinations = destinationsUnclean2.rename(columns={"End_lat": "lat", "End_lon": "lon", "End_station": "station", "End_station_number": "station_number"})
destinations = destinations.reset_index(drop=True)
destinations.head()

,lat,lon,station,station_number
0,51.490435,-0.122806,"Albert Embankment, Vauxhall",1059
1,51.527025,-0.088542,"Baldwin Street, St. Luke's",3500
2,51.485357,-0.142207,"Grosvenor Road, Pimlico",1140
3,51.488124,-0.120903,"Vauxhall Walk, Vauxhall",200056
4,51.490867,-0.116911,"Black Prince Road, Vauxhall",1205


In [140]:
# Create empty list to fill with corresponding bike end/start station
destinationPath = {}

for i in range(len(destinations)):
    updateDestination = destinations["station"][i]
    destinationPath.update({updateDestination: []})

In [141]:
# For every station in the list, note what the corresponding end/start station is for each ride
for path in destinationPath:

    for n in range(len(summary)):
        if summary["End_station"][n] == path:
            destinationPath[path] += [summary["Start_station"][n]]
        elif summary["Start_station"][n] == path:
            destinationPath[path] += [summary["End_station"][n]]

In [146]:
# For every station, count the number of times each end/start station occurs and take the top 5 for each.
topDestinationPath = []

for i in destinationPath:

    update = {"baseStation": i, "pathStation": [], "pathStationCount": []}
    count = pd.Series(destinationPath[i]).value_counts()
    top5 = count.to_frame().head(5).reset_index().rename(columns={"index": "station"})
    
    for n in range(len(top5)):
        update["pathStation"].append(top5["station"][n])
        update["pathStationCount"].append(int(top5["count"][n]))

    topDestinationPath.append(update)

In [148]:
with open('stationTop5Destination.json', 'w') as f:
    json.dump(topDestinationPath, f)